# Isabel's Jupyter NB: Homeownership

In [13]:
import pandas as pd
import numpy as np

### Format homeownership data to be numeric

In [81]:
# read in the data
homeownership = pd.read_csv("Homeownership.csv")
# check types
print("homeownership dftypes BEFORE\n",homeownership.dtypes)

# fix types to be numeric
homeownership[['TotalPopulation','OwnedMortgage','OwnedFree','RenterOccupied']]\
= homeownership.apply(lambda x: x[['TotalPopulation','OwnedMortgage','OwnedFree','RenterOccupied']].str.replace(',','').astype(float), axis=1)

print("\nhomeownership dftypes AFTER\n",homeownership.dtypes)
homeownership.head(10)

homeownership dftypes BEFORE
 Census Tract       float64
TotalPopulation     object
OwnedMortgage       object
OwnedFree           object
RenterOccupied      object
dtype: object

homeownership dftypes AFTER
 Census Tract       float64
TotalPopulation    float64
OwnedMortgage      float64
OwnedFree          float64
RenterOccupied     float64
dtype: object


,Census Tract,TotalPopulation,OwnedMortgage,OwnedFree,RenterOccupied
0,103.0,588.0,81.0,108.0,399.0
1,201.0,2538.0,304.0,168.0,2066.0
2,203.0,616.0,47.0,13.0,556.0
3,305.0,2140.0,410.0,151.0,1579.0
4,402.0,1345.0,297.0,142.0,906.0
5,404.0,2163.0,244.0,633.0,1286.0
6,405.0,2958.0,92.0,128.0,2738.0
7,406.0,2392.0,63.0,130.0,2199.0
8,409.0,2877.0,591.0,385.0,1901.0
9,501.0,1682.0,337.0,256.0,1089.0


### Format the neighborhood data to be numeric
#### (eventually relate census tract->neighborhood)

In [50]:
# read in neighborhoods dataset
neighborhood = pd.read_csv("neighborhood.csv")

print("dtype before :", neighborhood['tractce10'].dtype)

# utilize Pandas Series to convert all elements into one datatype
census_tract = pd.Series(neighborhood.iloc[:,4]) # index 4 is tractce10 column

# need to cast census tract nums (currently objects) into floats
# start by editing the format of each element
for i in range(len(census_tract)):
    string = census_tract[i]
    # if not empty string
    if (string != ''):
        firstChar = string[0]
        # run code only if the first char can be cast to a float
        try:
            float(firstChar)
            # if the firstChar is a number that starts with 0 as a placeholder (e.g., 024500)
            if (firstChar == '0' and len(string) > 1):
                # remove the first character
                string = string[1:]
            # remove extra zeros at the end
            census_tract[i] = string[0:len(string)-2]
            
        # first char cannot be converted to a float
        except ValueError:
            census_tract[i] = 0
    
    # if empty string
    else:
        census_tract[i] = 0
# end of for loop

# create dataframe and cast elements to float
censusTracts = pd.DataFrame(census_tract).astype(float, errors = 'raise')

print("dtype after:", censusTracts['tractce10'].dtypes)
censusTracts.head(10)

dtype before : object
dtype after: float64


,tractce10
0,1405.0
1,1401.0
2,1017.0
3,509.0
4,3001.0
5,1304.0
6,501.0
7,5628.0
8,404.0
9,1911.0


In [8]:
# add number of houses with mortgage and houses with mortgage payed off
mortgageHolders = homeownership['OwnedMortgage']
houseOwners = homeownership['OwnedFree']
homeOwners = mortgageHolders + houseOwners
# find ratio of renters because renting is associated with short term/transitional housing while owning a house/paying mortgage is
#^ characteristic of people (most likely families) settling down
renters = homeownership['RenterOccupied']
ratioSettled2Renting = homeOwners/renters
ratioSettled2Renting.head(4)

0    0.473684
1    0.228461
2    0.107914
3    0.355288
dtype: float64

In [10]:
# relate back to the census track (create/merge new dataframes)
ratio = pd.DataFrame(ratioSettled2Renting, columns=["Settled/renting"])
census_tracts = pd.DataFrame(homeownership['Census Tract'])
population = pd.DataFrame(homeownership['TotalPopulation'])

# edit census tract format

# create the new dataframe
my_df = census_tracts.join(ratio).join(population)
my_df.head(10)

,Census Tract,Settled/renting,TotalPopulation
0,103.0,0.473684,588.0
1,201.0,0.228461,2538.0
2,203.0,0.107914,616.0
3,305.0,0.355288,2140.0
4,402.0,0.484547,1345.0
5,404.0,0.681960,2163.0
6,405.0,0.080351,2958.0
7,406.0,0.087767,2392.0
8,409.0,0.513414,2877.0
9,501.0,0.544536,1682.0


In [11]:
# sort (we want the census tracts with the highest amount of house owners to renters)
my_dfsorted = my_df.sort_values(by=['Settled/renting'], ascending=False)
# get rid of any tract with less than 500 people
my_dfFiltered = my_dfsorted[my_dfsorted['TotalPopulation'] >= 500]

my_dfFiltered.head(10)

,Census Tract,Settled/renting,TotalPopulation
152,4268.00,43.741667,5369.0
244,4753.03,33.151261,4064.0
384,5641.00,32.923077,882.0
122,4100.00,30.648649,1171.0
237,4742.01,28.545455,2600.0
239,4742.03,27.306250,4529.0
125,4120.02,26.331461,4865.0
318,5190.00,23.680328,3011.0
149,4263.00,22.647059,6030.0
130,4134.00,21.978610,4297.0


In [ ]:
# No direct mapping from community neighborhoods to census tract. Census tracts follow county neighborhoods.
# Often times one neighborhood may contain many tracts and vice versa.
# The census tract numbers vary the least for regions geographically closest to each other 
#^ for example North, South, West, Central Oakland are 404,409,402,405.

# First get rid of decimal point